In [1]:
import pandas as pd

In [10]:
school_data = "./PyCitySchools/schools_complete.csv"

student_data = "./PyCitySchools/students_complete.csv"


In [11]:
school_df = pd.read_csv(school_data)
student_df = pd.read_csv(student_data)

In [39]:
merge_data = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])

merge_data

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## District Summary

### Totals for Schools, Students, and Budget

In [21]:
total_schools = len(merge_data['school_name'].unique())

total_students = merge_data['Student ID'].count()

total_budget = merge_data['budget'].sum()

### Average Math, and Reading Scores

In [22]:
average_math_score = merge_data['math_score'].mean()

average_reading_score = merge_data['reading_score'].mean()

### Percentage of Students Passing Math, Reading, and Math AND Reading

In [23]:
percent_pass_math = ((float(merge_data.loc[merge_data['math_score']>=70, ['math_score']].count())/ \
                     (float(merge_data.loc[merge_data['math_score'], ['math_score']].count()))*100))

percent_pass_read = ((float(merge_data.loc[merge_data['reading_score']>=70, ['reading_score']].count())/ \
                     (float(merge_data.loc[merge_data['reading_score'], ['reading_score']].count()))*100))

percent_pass_both = int(merge_data.loc[(merge_data['math_score']>=70) & \
                                    (merge_data['reading_score']>=70),['reading_score']].count())/ \
                                    total_students * 100

## School Summary

In [41]:
#Group by school name
groupedDF = merge_data.groupby(by='school_name')

#Total students and school budget
grouped_total_students = groupedDF.count()['student_name']
grouped_total_budget = groupedDF['budget'].unique()

#Per student budget
grouped_per_student_budget = grouped_total_budget / grouped_total_students

#Average math and reading scores
grouped_math = round(groupedDF.mean()['math_score'],2)
grouped_reading = round(groupedDF.mean()['reading_score'],2)

#Percent passing math, reading, and both
percent_pass_math = merge_data.loc[merge_data['math_score']>=70,['school_name','math_score']]
percent_pass_math = percent_pass_math.groupby(by='school_name')
percent_pass_math = percent_pass_math.count()
percent_pass_math['math_score'] = round(percent_pass_math['math_score'] / grouped_total_students * 100,2)

percent_pass_reading = merge_data.loc[merge_data['reading_score']>=70,['school_name','reading_score']]
percent_pass_reading = percent_pass_reading.groupby(by='school_name')
percent_pass_reading = percent_pass_reading.count()
percent_pass_reading['reading_score'] = round(percent_pass_reading['reading_score'] / grouped_total_students * 100,2) 

percent_pass_both = round((percent_pass_math['math_score']/100) * (percent_pass_reading['reading_score']/100) * 100,2)

#Concat dataframe
school_summaryDF = pd.DataFrame(data=[grouped_total_students, grouped_total_budget, grouped_per_student_budget, grouped_math,
                                     grouped_reading, percent_pass_math['math_score'], percent_pass_reading['reading_score'],
                                     percent_pass_both])

schoolTran = school_summaryDF.transpose()

schoolTran = schoolTran.rename(columns={'student_name': 'Total Students', 'budget': 'Total Budget', 
        'Unnamed 0': 'Per Student Budget', 'math_score': 'Math Score', 'reading': 'Reading Score', 
        'math_score': 'Percent Passing Math', 'reading_score': 'Percent Passing Reading', 'Unnamed 1': 'Percent Passing Both'})

schoolTran.head()

,Total Students,Total Budget,Per Student Budget,Percent Passing Math,Percent Passing Reading,Percent Passing Math,Percent Passing Reading,Percent Passing Both
school_name,,,,,,,,
Bailey High School,4976,[3124928],[628.0],77.05,81.03,66.68,81.93,54.63
Cabrera High School,1858,[1081356],[582.0],83.06,83.98,94.13,97.04,91.34
Figueroa High School,2949,[1884411],[639.0],76.71,81.16,65.99,80.74,53.28
Ford High School,2739,[1763916],[644.0],77.1,80.75,68.31,79.3,54.17
Griffin High School,1468,[917500],[625.0],83.35,83.82,93.39,97.14,90.72


## Highest-Performing Schools (by % Overall Passing)

In [29]:
top_schools = schoolTran.sort_values(['Percent Passing Both'], ascending=False)
top_schools = top_schools.head(5)

## Lowest-Performing Schools (by % Overall Passing)

In [30]:
bottom_schools = schoolTran.sort_values(['Percent Passing Both'], ascending=True)
bottom_schools = bottom_schools.head(5)

## Math Scores by Grade

In [31]:
mathDF = merge_data[['school_name','math_score']]
mathDF['grade'] = merge_data['grade'].apply(lambda x: x.split('t')[0])

mathDF = mathDF.groupby(['school_name','grade'])

math_per_school = round(mathDF.mean()['math_score'],2)

math_schoolDF = pd.DataFrame(data=[math_per_school])
math_schoolDF = math_schoolDF.transpose()

math_schoolDF.head()

C:\Users\Gamer\AppData\Local\Temp/ipykernel_13644/2720649223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mathDF['grade'] = merge_data['grade'].apply(lambda x: x.split('t')[0])


math_score
school_name         grade            
Bailey High School  10          77.00
                    11          77.52
                    12          76.49
                    9           77.08
Cabrera High School 10          83.15

## Reading Scores by Grade

In [32]:
readDF = merge_data[['school_name','reading_score']]
readDF['grade'] = merge_data['grade'].apply(lambda x: x.split('t')[0])

readDF = readDF.groupby(['school_name','grade'])

read_per_school = round(readDF.mean()['reading_score'],2)

read_schoolDF = pd.DataFrame(data=[read_per_school])
read_schoolDF = read_schoolDF.transpose()

read_schoolDF.head()

C:\Users\Gamer\AppData\Local\Temp/ipykernel_13644/1159226918.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  readDF['grade'] = merge_data['grade'].apply(lambda x: x.split('t')[0])


reading_score
school_name         grade               
Bailey High School  10             80.91
                    11             80.95
                    12             80.91
                    9              81.30
Cabrera High School 10             84.25

## Scores by School Spending

In [34]:
schoolTran.head(50)


bins = [578, 597.25, 616.5, 635.75, 655]
group_labels = ["578 to 597.24", "597.25 to 616.5", "616.51 to 635.75", "635.76 to 655"]

scores_by_school = pd.cut(schoolTran['Per Student Budget'], bins, labels=group_labels).head(20)

schoolTran['scores_by_school'] = scores_by_school

schoolTran

,Total Students,Total Budget,Per Student Budget,Percent Passing Math,Percent Passing Reading,Percent Passing Math,Percent Passing Reading,Percent Passing Both,scores_by_school
school_name,,,,,,,,,
Bailey High School,4976,[3124928],[628.0],77.05,81.03,66.68,81.93,54.63,616.51 to 635.75
Cabrera High School,1858,[1081356],[582.0],83.06,83.98,94.13,97.04,91.34,578 to 597.24
Figueroa High School,2949,[1884411],[639.0],76.71,81.16,65.99,80.74,53.28,635.76 to 655
Ford High School,2739,[1763916],[644.0],77.1,80.75,68.31,79.3,54.17,635.76 to 655
Griffin High School,1468,[917500],[625.0],83.35,83.82,93.39,97.14,90.72,616.51 to 635.75
Hernandez High School,4635,[3022020],[652.0],77.29,80.93,66.75,80.86,53.97,635.76 to 655
Holden High School,427,[248087],[581.0],83.8,83.81,92.51,96.25,89.04,578 to 597.24
Huang High School,2917,[1910635],[655.0],76.63,81.18,65.68,81.32,53.41,635.76 to 655
Johnson High School,4761,[3094650],[650.0],77.07,80.97,66.06,81.22,53.65,635.76 to 655


## Scores by School Size

In [37]:
bins = [427, 1943.33, 3459.66, 4976]
group_labels = ["small", "medium", "large"]

schoolTran["School Size"] = pd.cut(schoolTran["Total Students"], bins, labels=group_labels, right=False)
schoolTran

,Total Students,Total Budget,Per Student Budget,Percent Passing Math,Percent Passing Reading,Percent Passing Math,Percent Passing Reading,Percent Passing Both,scores_by_school,School Size
school_name,,,,,,,,,,
Bailey High School,4976,[3124928],[628.0],77.05,81.03,66.68,81.93,54.63,616.51 to 635.75,NaN
Cabrera High School,1858,[1081356],[582.0],83.06,83.98,94.13,97.04,91.34,578 to 597.24,small
Figueroa High School,2949,[1884411],[639.0],76.71,81.16,65.99,80.74,53.28,635.76 to 655,medium
Ford High School,2739,[1763916],[644.0],77.1,80.75,68.31,79.3,54.17,635.76 to 655,medium
Griffin High School,1468,[917500],[625.0],83.35,83.82,93.39,97.14,90.72,616.51 to 635.75,small
Hernandez High School,4635,[3022020],[652.0],77.29,80.93,66.75,80.86,53.97,635.76 to 655,large
Holden High School,427,[248087],[581.0],83.8,83.81,92.51,96.25,89.04,578 to 597.24,small
Huang High School,2917,[1910635],[655.0],76.63,81.18,65.68,81.32,53.41,635.76 to 655,medium
Johnson High School,4761,[3094650],[650.0],77.07,80.97,66.06,81.22,53.65,635.76 to 655,large
